# Demo Multiple Cutters - Multiple Barches
This notebook shows a basics of a feeder concept. This scenario has two vessels:
* a *vessel* for the transport of material
* an *installer*, receiving the material and installing it.
Thus, we also need two processes - one for each vessel. In this process the hand over of the material is done from the *vessel* to the *installer*.

In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import pandas as pd
import numpy as np

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import uuid

from plot import vessel_planning


# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of Sites

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie1",  # The name of the site
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 24,
    "level": 24,
    "nr_resources":3,
}  # The actual volume of the site
from_site = Site(**data_from_site)

location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "geometry": location_to_site,  # The coordinates of the project site
    "capacity": 24,
    "level": 0,
}  # The actual volume of the site (empty of course)
to_site = Site(**data_to_site)

init
level: 24
completed init
init
level: 0
completed init


## Definition of Vessels

In [3]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.ContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
        # SiteRegistry,
    ),
    {"key": "MultiStoreHopper"},
)

# print(SiteRegistry.inspect("MultiStoreHopper"))
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10

data_cutter1 = {
    "env": my_env,  # The simpy environment
    "name": "Cutter1",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}
cutter1 = TransportProcessingResource(**data_cutter1)
data_cutter2 = {
    "env": my_env,  # The simpy environment
    "name": "Cutter2",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}
cutter2 = TransportProcessingResource(**data_cutter2)

# TSHD variables
data_hopper1 = {
    "env": my_env,  # The simpy environment
    "name": "Hopper1",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}
hopper1 = TransportProcessingResource(**data_hopper1)
data_hopper2 = {
    "env": my_env,  # The simpy environment
    "name": "Hopper2",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}
hopper2 = TransportProcessingResource(**data_hopper2)
data_hopper3 = {
    "env": my_env,  # The simpy environment
    "name": "Hopper3",  # Name
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 1,
    "level": 0,
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}
hopper3 = TransportProcessingResource(**data_hopper3)

init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init


## Definition of Barge process

In [4]:
cutter_list = [cutter1, cutter2]
for hopper in [hopper1, hopper2, hopper3]:
    first_cutter = cutter_list[0]
    cutter_list= cutter_list[1:]
    cutter_list.append(first_cutter)
    for cutter in cutter_list:
        requested_resources = {}
        run = []
        
        shift_amount_loading_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "Load",  # We are moving soil
            "registry": registry,
            "processor": cutter,
            "origin": from_site,
            "destination": hopper,
            "amount": 1,
            "duration": 10,
            "requested_resources":requested_resources,
            "keep_resources":[hopper],
            "postpone_start": True,
        }
        run.append(model.ShiftAmountActivity(**shift_amount_loading_data))
        
        move_activity_to_harbor_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "sailing full",  # We are moving soil
            "registry": registry,
            "mover": hopper,
            "destination": to_site,
            "requested_resources":requested_resources,
            "keep_resources":[hopper],
            "postpone_start": True,
        }
        run.append(model.MoveActivity(**move_activity_to_harbor_data))

        shift_amount_loading_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "Unload",  # We are moving soil
            "registry": registry,
            "processor": hopper,
            "origin": hopper,
            "destination": to_site,
            "amount": 1,
            "duration": 10,
            "requested_resources":requested_resources,
            "keep_resources":[hopper],
            "postpone_start": True,
        }
        run.append(model.ShiftAmountActivity(**shift_amount_loading_data))
        
        move_activity_to_harbor_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "sailing empty",  # We are moving soil
            "registry": registry,
            "mover": hopper,
            "destination": from_site,
            "requested_resources":requested_resources,
            "postpone_start": True,
        }
        run.append(model.MoveActivity(**move_activity_to_harbor_data))

        sequential_activity_data = {
            "env": my_env,
            "name": "run",
            "registry": registry,
            "sub_processes": run,
            "postpone_start": True,
        }
        sequential_activity = model.SequentialActivity(**sequential_activity_data)
        
        while_data = {
            "env": my_env,  # The simpy environment defined in the first cel
            "name": "run while",  # We are moving soil
            "registry": registry,
            "sub_process": sequential_activity,
            "condition_event": [{"type":"container", "concept": from_site, "state":"empty"}],
            "postpone_start": False,
        }
        run_activity = model.WhileActivity(**while_data)

while Activity keep_resources []
start put_available
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None
while Activity keep_resources []
start put_available
check put_available events
register new event 
start event instance None


In [5]:
my_env.run()


conditional 
start event instance None
keep_resources []
SHift amount Activity keep_resources [<__main__.TransportProcessingResource object at 0x0000016B49FF2608>]
start event instance None
start : {}
{'origin.f3d40c02-abd3-11ea-a9c5-2079183a5c9a': 24, 'destination.f3d7de04-abd3-11ea-bd7f-2079183a5c9a': 1}
conditional 
start event instance None
keep_resources []
SHift amount Activity keep_resources [<__main__.TransportProcessingResource object at 0x0000016B49FF2608>]
start event instance None
start : {}
{'origin.f3d40c02-abd3-11ea-a9c5-2079183a5c9a': 24, 'destination.f3d7de04-abd3-11ea-bd7f-2079183a5c9a': 1}
conditional 
start event instance None
keep_resources []
SHift amount Activity keep_resources [<__main__.TransportProcessingResource object at 0x0000016B49FF2BC8>]
start event instance None
start : {}
{'origin.f3d40c02-abd3-11ea-a9c5-2079183a5c9a': 24, 'destination.f3d7de05-abd3-11ea-bee0-2079183a5c9a': 1}
conditional 
start event instance None
keep_resources []
SHift amount Activi

site request : {<simpy.resources.resource.Resource object at 0x0000016B49FF5448>: <Request() object at 0x16b4a00dd08>, <simpy.resources.resource.Resource object at 0x0000016B49FE9A48>: <Request() object at 0x16b4a00ee08>}
end requestIfAvailable : {<simpy.resources.resource.Resource object at 0x0000016B49FF5448>: <Request() object at 0x16b4a00dd08>, <simpy.resources.resource.Resource object at 0x0000016B49FE9A48>: <Request() object at 0x16b4a00ee08>}
after req resource if available : {<simpy.resources.resource.Resource object at 0x0000016B49FF5448>: <Request() object at 0x16b4a00dd08>, <simpy.resources.resource.Resource object at 0x0000016B49FE9A48>: <Request() object at 0x16b4a00ee08>}
_shift_amount
start _shift_amount
amount 1
processor start process
transfer default to Dumplocatie
processor process without rate
origin store before get: [{'id': 'default', 'level': 1, 'capacity': 1}]
start get 1
store_status {'id': 'default', 'level': 1, 'capacity': 1}
end get 1
get_callback - id_ <Sto

origin store after get: [{'id': 'default', 'level': 16, 'capacity': 24}]
after get
after check_down time
Mover_move after move
Activity end(sailing empty)
destination store before put: [{'id': 'default', 'level': 0, 'capacity': 1}]
<FilterStoreGet() object at 0x16b49b2dd48>
Activity end(run)
put_callback - id_ default
{}
destination store after put: [{'id': 'default', 'level': 1, 'capacity': 1}]
after put
processor end process
Processed 1 of default:
  by:          Cutter1
  origin        Winlocatie1  contains: 16 of default
  destination:  Hopper2 contains: 1 of default
after shift amount : {<simpy.resources.resource.Resource object at 0x0000016B49FF2E88>: <Request() object at 0x16b4a00e308>, <simpy.resources.resource.Resource object at 0x0000016B4A014EC8>: <Request() object at 0x16b4a00e7c8>, <simpy.resources.resource.Resource object at 0x0000016B49FE9548>: <Request() object at 0x16b49fff408>}
keep resources [<__main__.TransportProcessingResource object at 0x0000016B49FF2BC8>]
releas

start : {<simpy.resources.resource.Resource object at 0x0000016B49FF5448>: <Request() object at 0x16b4a00ec08>}
{'origin.f3d803a2-abd3-11ea-9387-2079183a5c9a': 1, 'destination.f3d4333e-abd3-11ea-9932-2079183a5c9a': 16}
put_callback - id_ default
{}
destination store after put: [{'id': 'default', 'level': 8, 'capacity': 24}]
after put
processor end process
Processed 1 of default:
  by:          Hopper2
  origin        Hopper2  contains: 0 of default
  destination:  Dumplocatie contains: 8 of default
after shift amount : {<simpy.resources.resource.Resource object at 0x0000016B49FF2E88>: <Request() object at 0x16b4a00e308>, <simpy.resources.resource.Resource object at 0x0000016B49FE9A48>: <Request() object at 0x16b4a00e148>}
keep resources [<__main__.TransportProcessingResource object at 0x0000016B49FF2BC8>]
release destination : {<simpy.resources.resource.Resource object at 0x0000016B49FF2E88>: <Request() object at 0x16b4a00e308>}
released origin : {<simpy.resources.resource.Resource obj

start event instance None
start : {<simpy.resources.resource.Resource object at 0x0000016B49FF28C8>: <Request() object at 0x16b4a014248>}
{'origin.f3d7de04-abd3-11ea-bd7f-2079183a5c9a': 1, 'destination.f3d4333e-abd3-11ea-9932-2079183a5c9a': 12}
destination request : {<simpy.resources.resource.Resource object at 0x0000016B49FF28C8>: <Request() object at 0x16b4a014248>, <simpy.resources.resource.Resource object at 0x0000016B49FE9A48>: <Request() object at 0x16b49ffae48>}
shift amount process keep_resources [<__main__.TransportProcessingResource object at 0x0000016B49FF2608>]
start get_available
processor request : {<simpy.resources.resource.Resource object at 0x0000016B49FF28C8>: <Request() object at 0x16b4a014248>, <simpy.resources.resource.Resource object at 0x0000016B49FE9A48>: <Request() object at 0x16b49ffae48>}
site request : {<simpy.resources.resource.Resource object at 0x0000016B49FF28C8>: <Request() object at 0x16b4a014248>, <simpy.resources.resource.Resource object at 0x0000016

start event instance None
Mover_move before mover resource request
Mover_move after mover resource request
Mover_move after move
Activity end(sailing empty)
Activity end(run)
destination request : {<simpy.resources.resource.Resource object at 0x0000016B49FF28C8>: <Request() object at 0x16b4a014308>}
shift amount process keep_resources [<__main__.TransportProcessingResource object at 0x0000016B49FF2608>]
start get_available
condition event triggered: False False round 2
while loop requested_resources {}
while loop keep_resources []
conditional 
start event instance None
keep_resources []
SHift amount Activity keep_resources [<__main__.TransportProcessingResource object at 0x0000016B49FF2608>]
start event instance None
start : {}
{'origin.f3d40c02-abd3-11ea-a9c5-2079183a5c9a': 9, 'destination.f3d7de04-abd3-11ea-bd7f-2079183a5c9a': 1}
processor request : {<simpy.resources.resource.Resource object at 0x0000016B49FF28C8>: <Request() object at 0x16b4a014308>, <simpy.resources.resource.Resour

destination store before put: [{'id': 'default', 'level': 0, 'capacity': 1}]
<FilterStoreGet() object at 0x16b4a03b188>
put_callback - id_ default
{}
destination store after put: [{'id': 'default', 'level': 1, 'capacity': 1}]
after put
processor end process
Processed 1 of default:
  by:          Cutter1
  origin        Winlocatie1  contains: 0 of default
  destination:  Hopper3 contains: 1 of default
after shift amount : {<simpy.resources.resource.Resource object at 0x0000016B49FF5448>: <Request() object at 0x16b4a0205c8>, <simpy.resources.resource.Resource object at 0x0000016B4A014EC8>: <Request() object at 0x16b4a048e88>, <simpy.resources.resource.Resource object at 0x0000016B49FE9548>: <Request() object at 0x16b4a048f48>}
keep resources [<__main__.TransportProcessingResource object at 0x0000016B49FF5188>]
release destination : {<simpy.resources.resource.Resource object at 0x0000016B49FF5448>: <Request() object at 0x16b4a0205c8>, <simpy.resources.resource.Resource object at 0x0000016

release destination : {<simpy.resources.resource.Resource object at 0x0000016B49FF2E88>: <Request() object at 0x16b49ff9d88>}
released origin : {<simpy.resources.resource.Resource object at 0x0000016B49FF2E88>: <Request() object at 0x16b49ff9d88>}
released processor : {<simpy.resources.resource.Resource object at 0x0000016B49FF2E88>: <Request() object at 0x16b49ff9d88>}
{<simpy.resources.resource.Resource object at 0x0000016B49FF2E88>: <Request() object at 0x16b49ff9d88>}
Activity end(Unload)
keep_resources []
start event instance None
Mover_move before mover resource request
Mover_move after mover resource request
destination request : {<simpy.resources.resource.Resource object at 0x0000016B49FF5448>: <Request() object at 0x16b4a0205c8>, <simpy.resources.resource.Resource object at 0x0000016B49FE9A48>: <Request() object at 0x16b4a03ba08>}
shift amount process keep_resources [<__main__.TransportProcessingResource object at 0x0000016B49FF5188>]
start get_available
processor request : {<

RuntimeError: Attempting to shift content from an empty origin or to a full destination. ({'origin.f3d40c02-abd3-11ea-a9c5-2079183a5c9a': 0, 'destination.f3d7de04-abd3-11ea-bd7f-2079183a5c9a': 1})

## Cutter1 log

In [ ]:
log_df = pd.DataFrame(cutter1.log)
data = log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data = data.drop_duplicates()
data


## Cutter2 log

In [ ]:
log2_df = pd.DataFrame(cutter2.log)
data2 = log2_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data2 = data2.drop_duplicates()
data2

## Hopper1 log

In [ ]:
hopper1_log_df = pd.DataFrame(hopper1.log)
data_hopper1 = hopper1_log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data_hopper1 = data_hopper1.drop_duplicates()
data_hopper1

## Hopper2 log

In [ ]:
hopper2_log_df = pd.DataFrame(hopper2.log)
data_hopper2 = hopper2_log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data_hopper2 = data_hopper2.drop_duplicates()
data_hopper2

## Hopper3 log

In [ ]:
hopper3_log_df = pd.DataFrame(hopper3.log)
data_hopper3 = hopper3_log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data_hopper3 = data_hopper3.drop_duplicates()
data_hopper3

In [6]:
objects = [cutter1, cutter2, hopper1, hopper2, hopper3]

activities = []
for obj in objects:
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

fig = vessel_planning(objects, activities)